In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [ ]:

#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

# Importing the warnings
import warnings
warnings.filterwarnings('ignore')



import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Pandas
import pandas as pd
# Remove Stopwords
from nltk.corpus import stopwords


# Import PorterStemmer from NLTK Library
from nltk.stem.porter import PorterStemmer
# Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.ensemble import ExtraTreesClassifier

from xgboost import XGBClassifier
# Metrix and Train Test
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


In [123]:
#df = pd.read_csv('/content/messages.csv')
# Train
df_train = pd.read_csv('mail_data.csv')

In [124]:
# Head Of Train Data
df_train.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [125]:
df_train.isnull().sum()

,0
Category,0
Message,0


In [126]:
# Null Values
print(f'Null Values in Train Dataset is : {df_train.isnull().sum().sum()}')

Null Values in Train Dataset is : 0


In [127]:
# Duplicates
print(f"Duplicates values in Train Dataset is : {df_train.duplicated().sum()}")

Duplicates values in Train Dataset is : 415


In [128]:
# Drop Duplicates
df_train.drop_duplicates(inplace=True)

In [129]:
# Null Values Columns
df_train.isnull().sum()

,0
Category,0
Message,0


In [130]:
# Lets Check Some Text
print(df_train['Message'][0])
print(df_train['Message'][1])
print(df_train['Message'][2])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Ok lar... Joking wif u oni...
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


Text Preprocessing

In This Section we do some Basis Text Preprocessing Like Cleaning Text , Tokenization Etc.

In [131]:
# 1. LowerCase
df_train['Message'] = df_train['Message'].str.lower()

# Head
df_train.head()

,Category,Message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [132]:
# 2. Remove # tag from Train Datasets
df_train['Message'] = df_train['Message'].str.replace('#','')

In [133]:
# 3. Remove @ From Train and Test Text
df_train['Message'] = df_train['Message'].str.replace('@','')

In [134]:
# 4. Remove URLs from Test and Train Text
df_train['Message'] = df_train['Message'].str.replace(r'^https?:\/\/.*[\r\n]*','')

In [135]:
# 5. Removing Punctuations
import string
df_train['Message'] = df_train['Message'].str.translate(str.maketrans('', '', string.punctuation))

# Head
df_train.head()

,Category,Message
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


In [136]:
# 6. Intilize Stopwords
stop_words = stopwords.words('english')

# Apply Stopwords
df_train['Message'] = df_train['Message'].apply(lambda x : ' '.join([word for word in x.split()if word not in (stop_words)]))

# Head
df_train.head()

,Category,Message
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah dont think goes usf lives around though


In [137]:
# Here Come ChatWords
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "B4N": "Bye For Now",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "ILU: I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don't care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can't stop laughing"
}

In [138]:
# 7. Handling ChatWords
# Function
def chat_conversion(text):
    new_text = []
    for i in text.split():
        if i.upper() in chat_words:
            new_text.append(chat_words[i.upper()])
        else:
            new_text.append(i)
    return " ".join(new_text)

# Calling Function
df_train['Message'] = df_train['Message'].apply(chat_conversion)

# Head
df_train.head()

,Category,Message
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif You oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,You dun say early hor You c already say
4,ham,nah dont think goes usf lives around though


In [139]:
# 8. Tokenization
from nltk.tokenize import sent_tokenize
import nltk

# Download the required resource
nltk.download('punkt_tab')

# Apply sent_tokenize
df_train['text_sent_token'] = df_train['Message'].apply(sent_tokenize)

# Head
df_train.head()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,Category,Message,text_sent_token
0,ham,go jurong point crazy available bugis n great ...,[go jurong point crazy available bugis n great...
1,ham,ok lar joking wif You oni,[ok lar joking wif You oni]
2,spam,free entry 2 wkly comp win fa cup final tkts 2...,[free entry 2 wkly comp win fa cup final tkts ...
3,ham,You dun say early hor You c already say,[You dun say early hor You c already say]
4,ham,nah dont think goes usf lives around though,[nah dont think goes usf lives around though]


In [141]:
# 9. Stemming

# Intilize Stemmer
stemmer = PorterStemmer()

# This Function Will Stem Words
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

# Calling
df_train['stem_msg'] = df_train['Message'].apply(stem_words)

# Head
df_train.head()

,Category,Message,text_sent_token,stem_msg
0,ham,go jurong point crazy available bugis n great ...,[go jurong point crazy available bugis n great...,go jurong point crazi avail bugi n great world...
1,ham,ok lar joking wif You oni,[ok lar joking wif You oni],ok lar joke wif you oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...,[free entry 2 wkly comp win fa cup final tkts ...,free entri 2 wkli comp win fa cup final tkt 21...
3,ham,You dun say early hor You c already say,[You dun say early hor You c already say],you dun say earli hor you c alreadi say
4,ham,nah dont think goes usf lives around though,[nah dont think goes usf lives around though],nah dont think goe usf live around though


Model Building

Text Representation / Converting text Into Numbers

In this Section , We Do Text Vectorization. Text Vectorization is the Process of Converting Text into Numbers

1.Initialization:

In [142]:
cv = CountVectorizer()

Fitting CountVectorizer on Text Data:

In [143]:
X = cv.fit_transform(df_train['stem_msg']).toarray()

In [144]:
# Intlize CountVectorizer
cv = CountVectorizer()

# Fitting CountVectorizer on X
X = cv.fit_transform(df_train['stem_msg']).toarray()

In [145]:
# Shape Of X
X.shape

(5157, 8082)

Encoding 'y'

In [146]:
# y
y = df_train['Category']

In [147]:
# Shape of Y
y.shape

(5157,)

In [148]:
# Intilize Enocder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Fitting Encoder
y = le.fit_transform(y)

# Y Values
y

array([0, 0, 1, ..., 0, 0, 0])

Train Test

In [149]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Model Fitting

Support Vector Machine (SVM):

In [150]:
svc = SVC(kernel='sigmoid', gamma=1.0)

2.K-Nearest Neighbors (KNN):

In [151]:
knc = KNeighborsClassifier()

3.Multinomial Naive Bayes:

In [152]:
mnb = MultinomialNB()

4.Decision Tree:

In [153]:
dtc = DecisionTreeClassifier(max_depth=5)

5.Logistic Regression:

In [154]:
lrc = LogisticRegression(solver='liblinear', penalty='l1')

6.Random Forest Classifier:

In [155]:
rfc = RandomForestClassifier(n_estimators=50, random_state=2)

7.AdaBoost Classifier:

In [156]:
abc = AdaBoostClassifier(n_estimators=50, random_state=2)

08.Extra Trees Classifier

In [157]:
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)

In [158]:
#9.XGBoost Classifier:
xgb = XGBClassifier(n_estimators=50, random_state=2)

Each model is now ready to be trained and evaluated on the dataset for the classification task. Adjust the hyperparameters as needed based on your specific task requirements and dataset characteristics.

In [159]:
# Initlize Models
# Support Vector MAchine
svc = SVC(kernel='sigmoid', gamma=1.0)
# KNeighbours
knc = KNeighborsClassifier()
# Multinomial NaiveBayes
mnb = MultinomialNB()
# Decision Tree
dtc = DecisionTreeClassifier(max_depth=5)
# Logistic Regression
lrc = LogisticRegression(solver='liblinear', penalty='l1')
# Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
# AddaBoost Classifier
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
# Extra Tree Classifier a Ensemble Method
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
# XGB Classifier
xgb = XGBClassifier(n_estimators=50,random_state=2)

The provided code fits each initialized model to the training data (X_train and y_train) and makes predictions on the test data (X_test). Here's a breakdown of the fitting and prediction process for each model:

In [160]:
#1.Support Vector Machine (SVC)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94       897
           1       0.64      0.59      0.61       135

    accuracy                           0.90      1032
   macro avg       0.79      0.77      0.78      1032
weighted avg       0.90      0.90      0.90      1032

[[852  45]
 [ 56  79]]


In [161]:
#2.K-Nearest Neighbors (KNN):
knc.fit(X_train, y_train)
knn_pred = knc.predict(X_test)

In [162]:
#3.Multinomial Naive Bayes:

mnb.fit(X_train, y_train)
mnb_pred = mnb.predict(X_test)

In [163]:
#4.Decision Tree:

dtc.fit(X_train, y_train)
dtc_pred = dtc.predict(X_test)

In [164]:
#5.Logistic Regression:

lrc.fit(X_train, y_train)
lrc_pred = lrc.predict(X_test)

In [165]:
#6.Random Forest Classifier:

rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

In [166]:
#7.AdaBoost Classifier:

abc.fit(X_train, y_train)
abc_pred = abc.predict(X_test)

In [167]:
#8.Extra Trees Classifier:

etc.fit(X_train, y_train)
etc_pred = etc.predict(X_test)

In [168]:
#9.XGBoost Classifier:

xgb = XGBClassifier(n_estimators=50, random_state=2)

In [169]:
# Initlize Models
# Support Vector MAchine
svc = SVC(kernel='sigmoid', gamma=1.0)
# KNeighbours
knc = KNeighborsClassifier()
# Multinomial NaiveBayes
mnb = MultinomialNB()
# Decision Tree
dtc = DecisionTreeClassifier(max_depth=5)
# Logistic Regression
lrc = LogisticRegression(solver='liblinear', penalty='l1')
# Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
# AddaBoost Classifier
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
# Extra Tree Classifier a Ensemble Method
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
# XGB Classifier
xgb = XGBClassifier(n_estimators=50,random_state=2)

The provided code fits each initialized model to the training data (X_train and y_train) and makes predictions on the test data (X_test). Here's a breakdown of the fitting and prediction process for each model:

In [170]:
#1.Support Vector Machine (SVC):

svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)

In [171]:
#2.K-Nearest Neighbors (KNN):

knc.fit(X_train, y_train)
knn_pred = knc.predict(X_test)

In [172]:
#3.Multinomial Naive Bayes:

mnb.fit(X_train, y_train)
mnb_pred = mnb.predict(X_test)

In [173]:
#4.Decision Tree:

dtc.fit(X_train, y_train)
dtc_pred = dtc.predict(X_test)

In [174]:
#5.Logistic Regression:

lrc.fit(X_train, y_train)
lrc_pred = lrc.predict(X_test)

In [175]:
#6.Random Forest Classifier:

rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

In [176]:
#7.AdaBoost Classifier:

abc.fit(X_train, y_train)
abc_pred = abc.predict(X_test)


In [177]:
#8.Extra Trees Classifier:

etc.fit(X_train, y_train)
etc_pred = etc.predict(X_test)

In [178]:
#9.XGBoost Classifier:

xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)